In [1]:
!pip install datasets
!pip install transformers

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/share/apps/python/3.8.6/intel/bin/python -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/share/apps/python/3.8.6/intel/bin/python -m pip install --upgrade pip' command.


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import DataLoader, Dataset
import collections
from transformers import RobertaTokenizer, RobertaForMaskedLM
from torch.utils.data.sampler import SubsetRandomSampler
from datasets import load_dataset
from data_loader_masked import EventSentenceLoader
from tqdm import tqdm
import re
import numpy as np
import argparse

In [3]:
use_freeze_model = False
validation_run = True
learning_rate = 0.00001

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')


def tokenize(batched_text):
    return tokenizer(batched_text['sentence'], padding=True, truncation=True, max_length=256)

In [5]:
class TextClassificationDataset(Dataset):
    def __init__(self, data):
            self.data_points = data

    def __len__(self):
        return len(self.data_points)

    def __getitem__(self, idx):
        data = self.data_points[idx]
        mask_id = tokenizer.mask_token_id
        new_tokens = torch.clone(data['tokens'])
        new_tokens = (new_tokens - (torch.ones(new_tokens.shape) * mask_id)) * (torch.ones(data['maskable'].shape) - data['maskable']) + (torch.ones(new_tokens.shape) * mask_id)
        new_tokens = new_tokens.long()
        old_tokens = data['tokens'].long()
        return {'labels': old_tokens.flatten(), 'attention': data['attention'].flatten(), 'tokens': new_tokens.flatten(), 'masked': data['maskable']}

In [6]:
def load_roberta_data():
    filepath = "new_sentences.txt"
    tokenizer_name = "roberta-base"
    loader = EventSentenceLoader(filepath, tokenizer_name)
    train_data = TextClassificationDataset(loader.load_data())

    train_size = int(0.9 * len(train_data))
    test_size = len(train_data) - train_size
    train_dataset, test_dataset = torch.utils.data.random_split(train_data, [train_size, test_size])
    train_data = list(map(lambda d: (torch.tensor(d['tokens']), d['labels'], d['attention'], d['masked']), train_dataset))
    test_data = list(map(lambda d: (torch.tensor(d['tokens']), d['labels'], d['attention'], d['masked']), test_dataset))

    return train_data, test_data

In [7]:
model = RobertaForMaskedLM.from_pretrained("roberta-base").to(device)

In [23]:
def test(model, data):
    confusion_matrix_size = 2
    confusion_matrix = []

    total = 0
    correct = 0
    total_real_words = 0

    for i in range(confusion_matrix_size):
        row = []
        for j in range(confusion_matrix_size):
            row.append(0)
        confusion_matrix.append(row)

    with torch.no_grad():
        total_loss = 0
        total_masked = 0
        wrong = 0
        for i, batch in tqdm(enumerate(data, 0), total=len(data), leave=False):
            inputs, labels, attention, mask = batch
            inputs, labels, attention, mask = inputs.to(device), labels.to(device), attention.to(device), mask.long().to(device)
            outputs = model(input_ids = inputs, attention_mask = attention, labels = labels)
            total_masked += mask.sum()
            predicted = outputs.logits.argmax(dim=-1)
            correct = labels.eq(predicted).sum()
            incorrect = labels.ne(predicted).sum()
            wrong += incorrect
            print(correct)
            
            loss = outputs.loss
            total_loss += loss.item()
        print((total_masked - wrong)/total_masked)
        
    return confusion_matrix

In [9]:
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
loss_func = nn.CrossEntropyLoss().to(device)

def train(model, data, epochs):
    n = len(data)

    # Define metrics to monitor change in performance during execution
    accuracy_history_epoch = []
    accuracy_history_step = []

    optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
    loss_func = nn.CrossEntropyLoss(weight=torch.tensor([0.1, 0.9])).to(device)

    for epoch in range(1, epochs + 1):
        correct = 0
        total = 0
        total_loss = 0
        for i, batch in tqdm(enumerate(data, 0), total=len(data), leave=False):
            inputs, labels, attention, mask = batch
            inputs, labels, attention, mask = inputs.to(device), labels.to(device), attention.to(device), mask.long().to(device)

            outputs = model(input_ids = inputs, attention_mask = attention, labels = labels)
            loss = outputs.loss
            total_loss += loss.item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print(total_loss)

    return accuracy_history_epoch, accuracy_history_step

In [10]:
filepath = "new_sentences.txt"
tokenizer_name = "roberta-base"
loader = EventSentenceLoader(filepath, tokenizer_name)
train_data = TextClassificationDataset(loader.load_data())

train_data, test_data = load_roberta_data()

<ipython-input-6-1fd493b7cd76>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_data = list(map(lambda d: (torch.tensor(d['tokens']), d['labels'], d['attention'], d['masked']), train_dataset))
<ipython-input-6-1fd493b7cd76>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_data = list(map(lambda d: (torch.tensor(d['tokens']), d['labels'], d['attention'], d['masked']), test_dataset))


In [11]:
indices = list(range(len(train_data)))

train_dataloader = DataLoader(train_data, batch_size=32, num_workers=0, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=32, num_workers=0, shuffle=0)

In [12]:
test(model, test_dataloader)

tensor(906, device='cuda:0')
tensor(37, device='cuda:0')
54.10102653503418


[[0, 0], [0, 0]]

In [15]:
train(model, train_dataloader, 250)

29.711336612701416


26.135920763015747


24.40906524658203


23.138280153274536


22.077983140945435


...

0.07065234938636422


0.06999180931597948


0.06943801511079073


0.06884032860398293


([], [])

In [24]:
test(model, test_dataloader)

tensor(8188, device='cuda:0')
tensor(256, device='cuda:0')
tensor(0.9259, device='cuda:0')


[[0, 0], [0, 0]]